In [25]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Configuração inicial do banco de dados
def configurar_banco():
    conexao = sqlite3.connect("sistema_vendas.db")
    cursor = conexao.cursor()

    # Tabelas necessárias
    cursor.execute("""CREATE TABLE IF NOT EXISTS usuarios (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        nome TEXT NOT NULL,
                        senha TEXT NOT NULL,
                        tipo TEXT NOT NULL)""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS produtos (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        nome TEXT NOT NULL,
                        fornecedor TEXT,
                        fabricante TEXT,
                        valor REAL NOT NULL)""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS clientes (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        nome TEXT NOT NULL,
                        cpf TEXT NOT NULL,
                        endereco TEXT NOT NULL,
                        cidade TEXT NOT NULL,
                        estado TEXT NOT NULL)""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS vendas (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        cliente_id INTEGER NOT NULL,
                        total REAL NOT NULL,
                        data TEXT NOT NULL,
                        FOREIGN KEY (cliente_id) REFERENCES clientes(id))""")
    
    cursor.execute("""CREATE TABLE IF NOT EXISTS itens_venda (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        venda_id INTEGER NOT NULL,
                        produto_id INTEGER NOT NULL,
                        quantidade INTEGER NOT NULL,
                        FOREIGN KEY (venda_id) REFERENCES vendas(id),
                        FOREIGN KEY (produto_id) REFERENCES produtos(id))""")

    # Inserir usuário administrador padrão
    cursor.execute("SELECT * FROM usuarios WHERE nome = 'admin'")
    if not cursor.fetchone():
        cursor.execute("INSERT INTO usuarios (nome, senha, tipo) VALUES (?, ?, ?)", 
                       ('admin', 'admin', 'administrador'))

    conexao.commit()
    conexao.close()

configurar_banco()

def tela_login():
    def autenticar():
        usuario = entry_usuario.get()
        senha = entry_senha.get()
        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("SELECT * FROM usuarios WHERE nome = ? AND senha = ?", (usuario, senha))
        resultado = cursor.fetchone()
        conexao.close()

        if resultado:
            login.destroy()
            abrir_painel(resultado[3])  # Tipo do usuário: "administrador" ou "comum"
        else:
            messagebox.showerror("Erro", "Usuário ou senha inválidos.")

    def abrir_cadastro():
        def atualizar_opcao_tipo_usuario(event):
            if tipo_usuario.get() == "Administrador":
                label_codigo.pack(pady=5)
                entry_codigo.pack(pady=5)
            else:
                label_codigo.pack_forget()
                entry_codigo.pack_forget()

        def cadastrar():
            novo_usuario = entry_novo_usuario.get()
            nova_senha = entry_nova_senha.get()
            tipo_selecionado = tipo_usuario.get()
            codigo_especial = entry_codigo.get()

            if not novo_usuario or not nova_senha or not tipo_selecionado:
                messagebox.showwarning("Atenção", "Preencha todos os campos obrigatórios.")
                return

            if tipo_selecionado == "Administrador" and codigo_especial != "manaconvite12":
                messagebox.showerror("Erro", "Código especial incorreto para administrador.")
                return

            conexao = sqlite3.connect("sistema_vendas.db")
            cursor = conexao.cursor()

            try:
                cursor.execute("""
                    INSERT INTO usuarios (nome, senha, tipo)
                    VALUES (?, ?, ?)
                """, (novo_usuario, nova_senha, tipo_selecionado.lower()))
                conexao.commit()
                messagebox.showinfo("Sucesso", "Usuário cadastrado com sucesso!")
                cadastro.destroy()
            except Exception as e:
                messagebox.showerror("Erro", f"Erro ao cadastrar usuário: {e}")
            finally:
                conexao.close()

        cadastro = tk.Toplevel(login)
        cadastro.title("Cadastro de Usuário")
        cadastro.geometry("300x500")

        tk.Label(cadastro, text="Novo Usuário:").pack(pady=5)
        entry_novo_usuario = tk.Entry(cadastro)
        entry_novo_usuario.pack(pady=5)

        tk.Label(cadastro, text="Nova Senha:").pack(pady=5)
        entry_nova_senha = tk.Entry(cadastro, show="*")
        entry_nova_senha.pack(pady=5)

        tk.Label(cadastro, text="Tipo de Usuário:").pack(pady=5)
        tipo_usuario = tk.StringVar(cadastro)
        tipo_usuario.set("Comum")  # Valor padrão
        menu_tipo_usuario = tk.OptionMenu(cadastro, tipo_usuario, "Comum", "Administrador", command=atualizar_opcao_tipo_usuario)
        menu_tipo_usuario.pack(pady=5)

        label_codigo = tk.Label(cadastro, text="Código Especial:")
        entry_codigo = tk.Entry(cadastro, show="*")

        tk.Button(cadastro, text="Cadastrar", command=cadastrar).pack(pady=10)
        tk.Button(cadastro, text="Cancelar", command=cadastro.destroy).pack()

    login = tk.Tk()
    login.title("Login - Sistema de Vendas")
    login.geometry("400x300")

    tk.Label(login, text="Usuário:").pack(pady=5)
    entry_usuario = tk.Entry(login)
    entry_usuario.pack(pady=5)

    tk.Label(login, text="Senha:").pack(pady=5)
    entry_senha = tk.Entry(login, show="*")
    entry_senha.pack(pady=5)

    tk.Button(login, text="Entrar", command=autenticar).pack(pady=10)
    tk.Button(login, text="Cadastrar", command=abrir_cadastro).pack(pady=5)
    tk.Button(login, text="Sair", command=login.destroy).pack()

    login.mainloop()

# Painel Principal
def abrir_painel(tipo_usuario):
    painel = tk.Tk()
    painel.title("Maná Conviteria Personalizada")
    painel.geometry("600x600")

    tk.Label(painel, text=f"Bem-vindo, {tipo_usuario.title()}!", font=("Arial", 14)).pack(pady=10)

    if tipo_usuario == "administrador":
        tk.Button(painel, text="Gerenciar Vendas", command=tela_vendas).pack(pady=5)
        tk.Button(painel, text="Cadastro de Produtos", command=tela_cadastro_produtos).pack(pady=5)
        tk.Button(painel, text="Cadastro de Clientes", command=tela_cadastro_clientes).pack(pady=5)
        tk.Button(painel, text="Relatório de Faturamento", command=tela_faturamento).pack(pady=5)
        tk.Button(painel, text="Visualizar Pedidos", command=tela_pedidos).pack(pady=5)
    else:
        tk.Button(painel, text="Gerenciar Vendas", command=tela_vendas).pack(pady=5)
        tk.Button(painel, text="Cadastro de Produtos", command=tela_cadastro_produtos).pack(pady=5)

    tk.Button(painel, text="Logout", command=painel.destroy).pack(pady=10)
    painel.mainloop()

# Executar a aplicação
configurar_banco()
tela_login()


# Painel Principal
def abrir_painel(tipo_usuario):
    painel = tk.Tk()
    painel.title("Maná Conviteria Personalizada")
    painel.geometry("600x600")

    tk.Label(painel, text=f"Bem-vindo, {tipo_usuario.title()}!", font=("Arial", 14)).pack(pady=10)

    if tipo_usuario == "administrador":
        tk.Button(painel, text="Gerenciar Vendas", command=tela_vendas).pack(pady=5)
        tk.Button(painel, text="Cadastro de Produtos", command=tela_cadastro_produtos).pack(pady=5)
        tk.Button(painel, text="Cadastro de Clientes", command=tela_cadastro_clientes).pack(pady=5)
        tk.Button(painel, text="Relatório de Faturamento", command=tela_faturamento).pack(pady=5)
        tk.Button(painel, text="Visualizar Pedidos", command=tela_pedidos).pack(pady=5)
    else:
        tk.Button(painel, text="Gerenciar Vendas", command=tela_vendas).pack(pady=5)
        tk.Button(painel, text="Cadastro de Produtos", command=tela_cadastro_produtos).pack(pady=5)

    tk.Button(painel, text="Logout", command=painel.destroy).pack(pady=10)
    painel.mainloop()

def tela_cadastro_produtos():
    def cadastrar_produto():
        nome = entry_nome.get()
        fornecedor = entry_fornecedor.get()
        fabricante = entry_fabricante.get()
        valor = entry_valor.get()

        if not nome or not valor:
            messagebox.showerror("Erro", "Nome e Valor são obrigatórios.")
            return

        try:
            valor = float(valor)
        except ValueError:
            messagebox.showerror("Erro", "Valor deve ser numérico.")
            return

        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("""
            INSERT INTO produtos (nome, fornecedor, fabricante, valor) 
            VALUES (?, ?, ?, ?)
        """, (nome, fornecedor, fabricante, valor))
        conexao.commit()
        conexao.close()

        messagebox.showinfo("Sucesso", "Produto cadastrado com sucesso!")
        carregar_produtos()  # Atualizar a tabela com os novos dados
        limpar_campos()

    def carregar_produtos():
        # Limpa a tabela
        for item in tree.get_children():
            tree.delete(item)

        # Carrega os dados do banco de dados
        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("SELECT id, nome, fornecedor, fabricante, valor FROM produtos")
        for row in cursor.fetchall():
            # Formata o valor como R$
            valor_formatado = f"R$ {row[4]:.2f}"
            tree.insert("", "end", values=(row[0], row[1], row[2], row[3], valor_formatado))
        conexao.close()

    def excluir_produto():
        item_selecionado = tree.selection()
        if not item_selecionado:
            messagebox.showwarning("Atenção", "Selecione um produto para excluir.")
            return

        resposta = messagebox.askyesno("Confirmação", "Tem certeza que deseja excluir o produto selecionado?")
        if resposta:
            # Obter o ID do item selecionado
            produto_id = tree.item(item_selecionado, "values")[0]

            # Remover o produto do banco de dados
            conexao = sqlite3.connect("sistema_vendas.db")
            cursor = conexao.cursor()
            cursor.execute("DELETE FROM produtos WHERE id = ?", (produto_id,))
            conexao.commit()
            conexao.close()

            # Atualizar a tabela
            carregar_produtos()
            messagebox.showinfo("Sucesso", "Produto excluído com sucesso!")

    def limpar_campos():
        entry_nome.delete(0, tk.END)
        entry_fornecedor.delete(0, tk.END)
        entry_fabricante.delete(0, tk.END)
        entry_valor.delete(0, tk.END)

    # Tela principal
    tela = tk.Toplevel()
    tela.title("Cadastro de Produtos")
    tela.geometry("700x500")

    # Campos de entrada
    tk.Label(tela, text="Nome do Produto:").pack(pady=5)
    entry_nome = tk.Entry(tela)
    entry_nome.pack(pady=5)

    tk.Label(tela, text="Fornecedor:").pack(pady=5)
    entry_fornecedor = tk.Entry(tela)
    entry_fornecedor.pack(pady=5)

    tk.Label(tela, text="Fabricante:").pack(pady=5)
    entry_fabricante = tk.Entry(tela)
    entry_fabricante.pack(pady=5)

    tk.Label(tela, text="Valor:").pack(pady=5)
    entry_valor = tk.Entry(tela)
    entry_valor.pack(pady=5)

    # Botões
    tk.Button(tela, text="Cadastrar", command=cadastrar_produto).pack(pady=10)
    tk.Button(tela, text="Excluir", command=excluir_produto).pack(pady=5)
    tk.Button(tela, text="Cancelar", command=tela.destroy).pack(pady=5)

    # Exibição dos produtos cadastrados
    tk.Label(tela, text="Produtos Cadastrados:").pack(pady=10)
    tree = ttk.Treeview(tela, columns=("ID", "Nome", "Fornecedor", "Fabricante", "Valor"), show="headings")
    tree.pack(fill="both", expand=True)

    # Configuração das colunas
    tree.heading("ID", text="ID")
    tree.heading("Nome", text="Nome")
    tree.heading("Fornecedor", text="Fornecedor")
    tree.heading("Fabricante", text="Fabricante")
    tree.heading("Valor", text="Valor")

    tree.column("ID", width=30, anchor="center")
    tree.column("Nome", width=150)
    tree.column("Fornecedor", width=120)
    tree.column("Fabricante", width=120)
    tree.column("Valor", width=80, anchor="center")

    # Carregar produtos no início
    carregar_produtos()

# Criação da tabela de produtos no banco de dados (execute uma vez)
def criar_tabela_produtos():
    conexao = sqlite3.connect("sistema_vendas.db")
    cursor = conexao.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS produtos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            fornecedor TEXT,
            fabricante TEXT,
            valor REAL NOT NULL
        )
    """)
    conexao.commit()
    conexao.close()

# Executar criação de tabela
criar_tabela_produtos()

# Gerenciamento de Vendas
def tela_vendas():
    def carregar_clientes():
        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("SELECT id, nome FROM clientes")
        clientes_cadastrados = cursor.fetchall()
        conexao.close()
        return clientes_cadastrados

    def carregar_produtos():
        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("SELECT id, nome, valor FROM produtos")
        produtos_cadastrados = cursor.fetchall()
        conexao.close()
        return produtos_cadastrados

    def adicionar_ao_carrinho():
        produto_nome = entry_produto_nome.get()
        produto_valor = entry_produto_valor.get()
        quantidade = entry_quantidade.get()

        if not produto_nome or not quantidade or not produto_valor:
            messagebox.showerror("Erro", "Nome do produto, valor e quantidade são obrigatórios.")
            return

        try:
            quantidade = int(quantidade)
            produto_valor = float(produto_valor)
        except ValueError:
            messagebox.showerror("Erro", "Quantidade e valor devem ser numéricos.")
            return

        item_total = produto_valor * quantidade
        carrinho.append((produto_nome, quantidade, produto_valor, item_total))
        atualizar_lista_carrinho()

    def atualizar_lista_carrinho():
        lista_carrinho.delete(0, tk.END)
        total = 0
        for item in carrinho:
            lista_carrinho.insert(tk.END, f"{item[0]} - Quantidade: {item[1]} - Total: R${item[3]:.2f}")
            total += item[3]
        label_total["text"] = f"Total da Venda: R${total:.2f}"

    def finalizar_venda():
        if not carrinho:
            messagebox.showerror("Erro", "Carrinho está vazio.")
            return

        cliente_nome = entry_cliente_nome.get()

        if not cliente_nome:
            messagebox.showerror("Erro", "Nome do cliente é obrigatório.")
            return

        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()

        # Inserir o cliente caso não exista
        cursor.execute("SELECT id FROM clientes WHERE nome = ?", (cliente_nome,))
        cliente = cursor.fetchone()
        if not cliente:
            cursor.execute("INSERT INTO clientes (nome, cpf, endereco, cidade, estado) VALUES (?, ?, ?, ?, ?)", 
                           (cliente_nome, "", "", "", ""))
            cliente_id = cursor.lastrowid
        else:
            cliente_id = cliente[0]

        total = sum(item[3] for item in carrinho)
        data_atual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute("INSERT INTO vendas (cliente_id, total, data) VALUES (?, ?, ?)", (cliente_id, total, data_atual))
        venda_id = cursor.lastrowid

        for item in carrinho:
            # Inserir produto caso não exista
            cursor.execute("SELECT id FROM produtos WHERE nome = ?", (item[0],))
            produto = cursor.fetchone()
            if not produto:
                cursor.execute("INSERT INTO produtos (nome, fornecedor, fabricante, valor) VALUES (?, ?, ?, ?)",
                               (item[0], "", "", item[2]))
                produto_id = cursor.lastrowid
            else:
                produto_id = produto[0]

            cursor.execute("INSERT INTO itens_venda (venda_id, produto_id, quantidade) VALUES (?, ?, ?)",
                           (venda_id, produto_id, item[1]))

        conexao.commit()
        conexao.close()

        messagebox.showinfo("Sucesso", "Venda finalizada com sucesso!")
        carrinho.clear()
        atualizar_lista_carrinho()

    carrinho = []

    tela = tk.Toplevel()
    tela.title("Gerenciamento de Vendas")
    tela.geometry("500x700")

    # Carregar dados
    clientes = carregar_clientes()
    produtos = carregar_produtos()

    # Seleção de cliente
    tk.Label(tela, text="Nome do Cliente:").pack(pady=5)
    entry_cliente_nome = tk.Entry(tela)
    entry_cliente_nome.pack(pady=5)

    tk.Label(tela, text="Selecione Cliente:").pack(pady=5)
    clientes_var = tk.StringVar(tela)
    clientes_var.set("Selecionar cliente")
    tk.OptionMenu(tela, clientes_var, *[cliente[1] for cliente in clientes]).pack(pady=5)
    clientes_var.trace("w", lambda *args: entry_cliente_nome.delete(0, tk.END) or entry_cliente_nome.insert(0, clientes_var.get()))

    # Seleção de produto
    tk.Label(tela, text="Nome do Produto:").pack(pady=5)
    entry_produto_nome = tk.Entry(tela)
    entry_produto_nome.pack(pady=5)

    tk.Label(tela, text="Valor do Produto:").pack(pady=5)
    entry_produto_valor = tk.Entry(tela)
    entry_produto_valor.pack(pady=5)

    tk.Label(tela, text="Selecione Produto:").pack(pady=5)
    produtos_var = tk.StringVar(tela)
    produtos_var.set("Selecionar produto")
    tk.OptionMenu(tela, produtos_var, *[produto[1] for produto in produtos]).pack(pady=5)
    produtos_var.trace(
        "w", lambda *args: (
            entry_produto_nome.delete(0, tk.END) or 
            entry_produto_valor.delete(0, tk.END) or 
            entry_produto_nome.insert(0, produtos_var.get()) or 
            entry_produto_valor.insert(0, str(next((produto[2] for produto in produtos if produto[1] == produtos_var.get()), 0)))
        )
    )

    tk.Label(tela, text="Quantidade:").pack(pady=5)
    entry_quantidade = tk.Entry(tela)
    entry_quantidade.pack(pady=5)

    tk.Button(tela, text="Adicionar ao Carrinho", command=adicionar_ao_carrinho).pack(pady=10)

    lista_carrinho = tk.Listbox(tela, width=50, height=10)
    lista_carrinho.pack(pady=10)

    label_total = tk.Label(tela, text="Total da Venda: R$0.00", font=("Arial", 12))
    label_total.pack(pady=5)

    tk.Button(tela, text="Finalizar Venda", command=finalizar_venda).pack(pady=10)
    tk.Button(tela, text="Cancelar", command=tela.destroy).pack()

# Cadastro de Clientes
def tela_cadastro_clientes():
    def cadastrar_cliente():
        nome = entry_nome.get()
        cpf = entry_cpf.get()
        endereco = entry_endereco.get()
        cidade = entry_cidade.get()
        estado = entry_estado.get()

        if not nome or not cpf or not endereco or not cidade or not estado:
            messagebox.showerror("Erro", "Todos os campos são obrigatórios.")
            return

        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("INSERT INTO clientes (nome, cpf, endereco, cidade, estado) VALUES (?, ?, ?, ?, ?)", 
                       (nome, cpf, endereco, cidade, estado))
        conexao.commit()
        conexao.close()

        messagebox.showinfo("Sucesso", "Cliente cadastrado com sucesso!")
        tela.destroy()

    tela = tk.Toplevel()
    tela.title("Cadastro de Clientes")
    tela.geometry("400x500")

    tk.Label(tela, text="Nome do Cliente:").pack(pady=5)
    entry_nome = tk.Entry(tela)
    entry_nome.pack(pady=5)

    tk.Label(tela, text="CPF:").pack(pady=5)
    entry_cpf = tk.Entry(tela)
    entry_cpf.pack(pady=5)

    tk.Label(tela, text="Endereço:").pack(pady=5)
    entry_endereco = tk.Entry(tela)
    entry_endereco.pack(pady=5)

    tk.Label(tela, text="Cidade:").pack(pady=5)
    entry_cidade = tk.Entry(tela)
    entry_cidade.pack(pady=5)

    tk.Label(tela, text="Estado:").pack(pady=5)
    entry_estado = tk.Entry(tela)
    entry_estado.pack(pady=5)

    tk.Button(tela, text="Cadastrar", command=cadastrar_cliente).pack(pady=10)
    tk.Button(tela, text="Cancelar", command=tela.destroy).pack()

def tela_faturamento():
    def carregar_dados():
        data_inicial = entry_data_inicial.get()
        data_final = entry_data_final.get()

        try:
            # Validar as datas
            data_inicial = datetime.strptime(data_inicial, "%Y-%m-%d")
            data_final = datetime.strptime(data_final, "%Y-%m-%d")
        except ValueError:
            messagebox.showerror("Erro", "Datas inválidas! Use o formato YYYY-MM-DD.")
            return

        if data_inicial > data_final:
            messagebox.showerror("Erro", "A data inicial deve ser menor ou igual à data final.")
            return

        # Consultar os dados no banco
        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("""
            SELECT 
                strftime('%Y-%m', data) AS mes,
                SUM(total) AS faturamento
            FROM vendas
            WHERE data BETWEEN ? AND ?
            GROUP BY mes
            ORDER BY mes
        """, (data_inicial, data_final))

        dados = cursor.fetchall()
        conexao.close()

        # Atualizar o gráfico e a tabela
        atualizar_grafico(dados)
        atualizar_tabela(data_inicial, data_final)

    def atualizar_grafico(dados):
        meses = [dado[0] for dado in dados]
        faturamentos = [dado[1] for dado in dados]

        # Limpar gráfico anterior
        ax.clear()
        ax.bar(meses, faturamentos, color='blue')
        ax.set_title("Faturamento por Mês")
        ax.set_xlabel("Mês")
        ax.set_ylabel("Faturamento (R$)")
        canvas.draw()

    def atualizar_tabela(data_inicial, data_final):
        # Limpar a tabela
        for item in tree.get_children():
            tree.delete(item)

        # Carregar vendas detalhadas no período
        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("""
            SELECT 
                vendas.id,
                clientes.nome,
                vendas.total,
                vendas.data
            FROM vendas
            JOIN clientes ON vendas.cliente_id = clientes.id
            WHERE vendas.data BETWEEN ? AND ?
            ORDER BY vendas.data
        """, (data_inicial, data_final))

        vendas = cursor.fetchall()
        conexao.close()

        # Preencher a tabela
        for venda in vendas:
            tree.insert("", "end", values=venda)

    # Criar a janela
    tela = tk.Toplevel()
    tela.title("Relatório de Faturamento")
    tela.geometry("800x600")

    # Filtros de Data
    frame_filtros = tk.Frame(tela)
    frame_filtros.pack(pady=10)

    tk.Label(frame_filtros, text="Data Inicial (YYYY-MM-DD):").grid(row=0, column=0, padx=5)
    entry_data_inicial = tk.Entry(frame_filtros)
    entry_data_inicial.grid(row=0, column=1, padx=5)

    tk.Label(frame_filtros, text="Data Final (YYYY-MM-DD):").grid(row=0, column=2, padx=5)
    entry_data_final = tk.Entry(frame_filtros)
    entry_data_final.grid(row=0, column=3, padx=5)

    tk.Button(frame_filtros, text="Aplicar Filtros", command=carregar_dados).grid(row=0, column=4, padx=10)

    # Gráfico de Faturamento
    frame_grafico = tk.Frame(tela)
    frame_grafico.pack(fill="both", expand=True)

    figura = plt.Figure(figsize=(6, 4), dpi=100)
    ax = figura.add_subplot(111)
    canvas = FigureCanvasTkAgg(figura, master=frame_grafico)
    canvas.get_tk_widget().pack(fill="both", expand=True)

    # Tabela de Vendas
    frame_tabela = tk.Frame(tela)
    frame_tabela.pack(fill="both", expand=True)

    tree = ttk.Treeview(frame_tabela, columns=("ID", "Cliente", "Total", "Data"), show="headings")
    tree.pack(fill="both", expand=True)

    tree.heading("ID", text="ID")
    tree.heading("Cliente", text="Cliente")
    tree.heading("Total", text="Total (R$)")
    tree.heading("Data", text="Data")

    tree.column("ID", width=50)
    tree.column("Cliente", width=200)
    tree.column("Total", width=100, anchor="center")
    tree.column("Data", width=100, anchor="center")

def tela_pedidos():
    def carregar_pedidos():
        """Carrega os pedidos na tabela com base nos filtros aplicados."""
        filtro_cliente = entry_cliente.get()
        filtro_status = combo_status.get()
        filtro_data = entry_data.get()

        # Construir consulta SQL dinâmica
        query = """
            SELECT pedidos.id, clientes.nome, pedidos.data, pedidos.status
            FROM pedidos
            JOIN clientes ON pedidos.cliente_id = clientes.id
            WHERE 1=1
        """
        parametros = []

        if filtro_cliente:
            query += " AND clientes.nome LIKE ?"
            parametros.append(f"%{filtro_cliente}%")
        
        if filtro_status and filtro_status != "Todos":
            query += " AND pedidos.status = ?"
            parametros.append(filtro_status)
        
        if filtro_data:
            query += " AND pedidos.data = ?"
            parametros.append(filtro_data)

        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute(query, parametros)
        pedidos = cursor.fetchall()
        conexao.close()

        # Limpar tabela antes de adicionar novos dados
        for item in tree_pedidos.get_children():
            tree_pedidos.delete(item)

        # Inserir os pedidos na tabela
        for pedido in pedidos:
            tree_pedidos.insert("", "end", values=pedido)

    def exibir_detalhes(event):
        """Exibe os detalhes do pedido selecionado."""
        selecionado = tree_pedidos.selection()
        if not selecionado:
            return

        pedido_id = tree_pedidos.item(selecionado, "values")[0]

        conexao = sqlite3.connect("sistema_vendas.db")
        cursor = conexao.cursor()
        cursor.execute("""
            SELECT produtos.nome, itens_pedido.quantidade, itens_pedido.preco_unitario,
                   itens_pedido.quantidade * itens_pedido.preco_unitario AS total
            FROM itens_pedido
            JOIN produtos ON itens_pedido.produto_id = produtos.id
            WHERE itens_pedido.pedido_id = ?
        """, (pedido_id,))
        detalhes = cursor.fetchall()
        conexao.close()

        # Limpar tabela de detalhes
        for item in tree_detalhes.get_children():
            tree_detalhes.delete(item)

        # Inserir os detalhes na tabela
        for detalhe in detalhes:
            tree_detalhes.insert("", "end", values=detalhe)

    # Criar janela
    tela = tk.Toplevel()
    tela.title("Tela de Pedidos")
    tela.geometry("900x600")

    # Filtros
    frame_filtros = tk.Frame(tela)
    frame_filtros.pack(pady=10, fill="x")

    tk.Label(frame_filtros, text="Cliente:").grid(row=0, column=0, padx=5)
    entry_cliente = tk.Entry(frame_filtros)
    entry_cliente.grid(row=0, column=1, padx=5)

    tk.Label(frame_filtros, text="Status:").grid(row=0, column=2, padx=5)
    combo_status = ttk.Combobox(frame_filtros, values=["Todos", "Pendente", "Finalizado", "Cancelado"])
    combo_status.set("Todos")
    combo_status.grid(row=0, column=3, padx=5)

    tk.Label(frame_filtros, text="Data (YYYY-MM-DD):").grid(row=0, column=4, padx=5)
    entry_data = tk.Entry(frame_filtros)
    entry_data.grid(row=0, column=5, padx=5)

    tk.Button(frame_filtros, text="Filtrar", command=carregar_pedidos).grid(row=0, column=6, padx=10)

    # Tabela de Pedidos
    frame_pedidos = tk.Frame(tela)
    frame_pedidos.pack(fill="both", expand=True)

    tree_pedidos = ttk.Treeview(frame_pedidos, columns=("ID", "Cliente", "Data", "Status"), show="headings")
    tree_pedidos.pack(fill="both", expand=True)

    tree_pedidos.heading("ID", text="ID")
    tree_pedidos.heading("Cliente", text="Cliente")
    tree_pedidos.heading("Data", text="Data")
    tree_pedidos.heading("Status", text="Status")

    tree_pedidos.column("ID", width=50)
    tree_pedidos.column("Cliente", width=200)
    tree_pedidos.column("Data", width=100, anchor="center")
    tree_pedidos.column("Status", width=100, anchor="center")

    # Conectar evento para exibir detalhes
    tree_pedidos.bind("<Double-1>", exibir_detalhes)

    # Tabela de Detalhes do Pedido
    frame_detalhes = tk.LabelFrame(tela, text="Detalhes do Pedido")
    frame_detalhes.pack(fill="both", expand=True, pady=10)

    tree_detalhes = ttk.Treeview(frame_detalhes, columns=("Produto", "Quantidade", "Preço Unitário", "Total"), show="headings")
    tree_detalhes.pack(fill="both", expand=True)

    tree_detalhes.heading("Produto", text="Produto")
    tree_detalhes.heading("Quantidade", text="Quantidade")
    tree_detalhes.heading("Preço Unitário", text="Preço Unitário (R$)")
    tree_detalhes.heading("Total", text="Total (R$)")

    tree_detalhes.column("Produto", width=200)
    tree_detalhes.column("Quantidade", width=100, anchor="center")
    tree_detalhes.column("Preço Unitário", width=150, anchor="center")
    tree_detalhes.column("Total", width=150, anchor="center")

# Inicialização
if __name__ == "__main__":
    tela_login()